# NLP

In [2]:
import pandas as pd 
import re
import sys
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors

In [3]:
df = pd.read_csv('data.csv')
df.head()

,text,domain,label
0,Thủ tướng Abe cúi đầu xin lỗi vì hành động phi...,binhluan.biz,1
1,Thủ tướng Nhật cúi đầu xin lỗi vì tinh thần ph...,www.ipick.vn,1
2,Choáng! Cơ trưởng đeo khăn quàng quẩy banh nóc...,tintucqpvn.net,1
3,Chưa bao giờ nhạc Kpop lại dễ hát đến thế!!!\n...,tintucqpvn.net,1
4,"Đại học Hutech sẽ áp dụng cải cách ""Tiếq Việt""...",www.gioitreviet.net,1


In [4]:
# !pip install underthesea

In [5]:
df1 = df.copy()

# Tiền xử lý

## Lower case

In [6]:
df1['text'] = df1['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [7]:
df1.head()

,text,domain,label
0,thủ tướng abe cúi đầu xin lỗi vì hành động phi...,binhluan.biz,1
1,thủ tướng nhật cúi đầu xin lỗi vì tinh thần ph...,www.ipick.vn,1
2,choáng! cơ trưởng đeo khăn quàng quẩy banh nóc...,tintucqpvn.net,1
3,chưa bao giờ nhạc kpop lại dễ hát đến thế!!! g...,tintucqpvn.net,1
4,"đại học hutech sẽ áp dụng cải cách ""tiếq việt""...",www.gioitreviet.net,1


## Remove stopwords

file **vietnamese_stopwords.txt** lấy từ [link github](https://github.com/stopwords/vietnamese-stopwords/blob/master/vietnamese-stopwords.txt)

In [8]:
def get_stopwords_list(stop_file_path):
    """load stop words """
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))
vietnamese = get_stopwords_list('vietnamese_stopwords.txt')

In [9]:
len(vietnamese)
df1.head()


,text,domain,label
0,thủ tướng abe cúi đầu xin lỗi vì hành động phi...,binhluan.biz,1
1,thủ tướng nhật cúi đầu xin lỗi vì tinh thần ph...,www.ipick.vn,1
2,choáng! cơ trưởng đeo khăn quàng quẩy banh nóc...,tintucqpvn.net,1
3,chưa bao giờ nhạc kpop lại dễ hát đến thế!!! g...,tintucqpvn.net,1
4,"đại học hutech sẽ áp dụng cải cách ""tiếq việt""...",www.gioitreviet.net,1


In [10]:
temp = vietnamese.copy()
temp.sort()
temp.reverse()

In [11]:
with open('temp.txt','w',encoding="utf-8") as out:
    for i in temp:
        out.write(i+'\n')
        


In [12]:
temp = vietnamese.copy()
temp.sort()
temp.reverse()

def drop_stopwords(text):
    text_list = text.split()
    text_len = len(text_list)
    for word in temp:
        word_list = word.split()
        word_len = len(word_list)
        for i in range(text_len + 1 - word_len):
            if text_list[i:i+word_len] == word_list:
                text_list[i:i+word_len] = [None] * word_len
    return ' '.join(t for t in text_list if t)

In [13]:
drop_stopwords(df1.text[0])

'thủ tướng abe cúi đầu lỗi hành động phi thể thao tuyển nhật sankei sports, thủ tướng nhật shinzo abe công khai gửi lỗi nhật hoàng toàn dân tinh thần thi đấu phi thể thao đội tuyển nhật world cup 2018. tối qua, lượt trận vòng bảng world cup 2018, nhật phụ senegal, đội thua colombia. nhật vòng fair-play thẻ phạt hơn. thủ tướng nhật shinzo abe cúi đầu lỗi. “với tinh thần võ sĩ đạo samurai, đội tuyển nhật hành động tinh thần thượng võ thi đấu world cup, trích bất bình toàn dân, hình ảnh kiên cường dân nhật đấu trường quốc tế. đứng đầu, thành trách nhiệm gửi lỗi sâu sắc nhân dân.”, abe cúi đầu trách nhiệm mình. nhật nổi tiếng tinh thần, trách nhiệm chất võ sĩ đạo. đội tuyển áo xanh mệnh danh “samurai xanh”. nhiên, nhật trích dội hâm mộ bóng đá lối bóng tiêu cực, cố tình câu vé đi tiếp. cụ thể, phút trận vòng bảng lan, trận giờ, colombia thắng senegal tỷ 1-0 tỉ tương tự, cầu thủ nhật hề gỡ hòa. cầu thủ nhật vui vẻ trận thua lan 0-1, vé vòng 16 đội. world cup 2018, fifa fair play, quy định 1

In [14]:

df1['text'] = df1['text'].apply(drop_stopwords)

In [15]:
df1.head()

,text,domain,label
0,thủ tướng abe cúi đầu lỗi hành động phi thể th...,binhluan.biz,1
1,thủ tướng nhật cúi đầu lỗi tinh thần phi thể t...,www.ipick.vn,1
2,choáng! trưởng đeo khăn quàng quẩy banh sân kh...,tintucqpvn.net,1
3,bao nhạc kpop hát thế!!! giáo sư nguyễn lân dũ...,tintucqpvn.net,1
4,"đại học hutech áp dụng cải ""tiếq việt"" học 201...",www.gioitreviet.net,1


## Remove noise
(HTML tag, các ký hiệu đặc biệt như @, #,...)

In [ ]:
TAG_1 = re.compile(r'<[^>]+>')
TAG_2 = re.compile("[@#$]")
df2 = df1.copy()
def remove_tags(text):
    text = TAG_1.sub('', text)
    return TAG_2.sub('', text)
df2['text'] = df2['text'].apply(remove_tags)


In [ ]:
df2['text']

## Remove punctuation (loại bỏ dấu câu)

In [ ]:
df1['text'] = df1['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

In [ ]:
df1['text'][0]

In [ ]:
from underthesea import pos_tag
pos_tag(df1['text'][0])

In [ ]:
label = df1.label
conversion_dict = {0:'Real', 1:'Fake'}
label = label.replace(conversion_dict)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df1.text,label,test_size=0.3,random_state=17)

In [ ]:
tfidf_vectorizer=TfidfVectorizer(max_df=0.7)
tfidf_train=tfidf_vectorizer.fit_transform(x_train.values.astype('U'))
tfidf_test=tfidf_vectorizer.transform(x_test.values.astype('U'))

PAC


In [ ]:
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train,y_train)



In [ ]:
y_pred=pac.predict(tfidf_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

In [ ]:
confusion_matrix(y_test,y_pred,labels=['Real','Fake'])

In [ ]:
X= tfidf_vectorizer.transform(df1.text.values.astype('U'))

In [ ]:
scores = cross_val_score(pac, X, df['label'].values,cv=5)
print(f'K Fold Accuracy: {round(scores.mean()*100,2)}%')

In [ ]:
def findlabel(text):
    vec_test =  tfidf_vectorizer.transform([text])
    y_pred = pac.predict(vec_test)
    return y_pred[0]

In [ ]:
findlabel(df1.text[100])

CLF

In [ ]:

clf = KNeighborsClassifier()
clf.fit(tfidf_train,y_train)

In [ ]:
y_pred=clf.predict(tfidf_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

In [ ]:
confusion_matrix(y_test,y_pred,labels=['Real','Fake'])

In [ ]:
X= tfidf_vectorizer.transform(df1.text.values.astype('U'))

In [ ]:
scores = cross_val_score(clf, X, df['label'].values,cv=5)
print(f'K Fold Accuracy: {round(scores.mean()*100,2)}%')

In [ ]:
def findlabel1(text):
    vec_test =  tfidf_vectorizer.transform([text])
    y_pred = clf.predict(vec_test)
    return y_pred[0]

In [ ]:
findlabel1(df1.text[100])